In [1]:
import os
import sys
import pickle
import psutil
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Define data path
DATA_PATH = "data/"
CHECKPOINT_PATH = "models/"

In [2]:
pids = pickle.load(open(os.path.join(DATA_PATH,'pids.pkl'), 'rb'))
vids = pickle.load(open(os.path.join(DATA_PATH,'vids.pkl'), 'rb'))
targets = pickle.load(open(os.path.join(DATA_PATH,'targets.pkl'), 'rb'))
prob_targets = pickle.load(open(os.path.join(DATA_PATH,'prob_targets.pkl'), 'rb'))
prob_targets_allvisits = pickle.load(open(os.path.join(DATA_PATH,'prob_targets_allvisits.pkl'), 'rb'))
seqs = pickle.load(open(os.path.join(DATA_PATH,'seqs.pkl'), 'rb'))
diags = pickle.load(open(os.path.join(DATA_PATH,'diags.pkl'), 'rb'))
categories = pickle.load(open(os.path.join(DATA_PATH,'categories.pkl'), 'rb'))
sub_categories = pickle.load(open(os.path.join(DATA_PATH,'subcategories.pkl'), 'rb'))
codes = pickle.load(open(os.path.join(DATA_PATH,'icd9.pkl'), 'rb'))
assert len(pids) == len(vids) == len(targets) == len(seqs)

In [3]:
class CustomDataset(Dataset):
    
    def __init__(self, seqs, targets):
        self.x = seqs
        self.y = targets
    
    def __len__(self):
        return(len(self.x))
    
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

In [4]:
dataset = CustomDataset(seqs, prob_targets_allvisits)

In [5]:
def collate_fn(data):
    """
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (# total visits excluding last visit per patient, max # diagnosis codes)
            of type torch.long
        x_masks: a tensor of shape (# total visits excluding last visit per patient, max # diagnosis codes)
            of type torch.bool
        y: a tensor of shape (# total visits excluding first visit per patient, num higher level categories
            to predict) of type torch.float
    """
    sequences, targets = zip(*data)
    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]
    num_categories = len(targets[0][0])

    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
    
    sum_visits = sum(num_visits)
    
    x = torch.zeros((sum_visits - num_patients, max_num_codes), dtype=torch.int)
    y = torch.zeros((sum_visits - num_patients, num_categories), dtype=torch.float32)
    x_masks = torch.zeros((sum_visits - num_patients, max_num_codes), dtype=torch.bool)

    n = 0
    for i,patient in enumerate(sequences):
        for j,visit in enumerate(patient):
            if j == len(patient) - 1:
                break
            for k,code in enumerate(visit):
                x[n,k] = code
                x_masks[n,k] = 1
            n+=1
    n = 0
    for i,patient in enumerate(targets):
        for j,visit in enumerate(patient):
            if j == len(patient) - 1:
                break
            y[n] = torch.tensor(patient[j+1])
            n += 1
    
    return x, x_masks, y

In [6]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)
val_split = int(len(dataset)*0.10)

In [7]:
from torch.utils.data.dataset import random_split

train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)

lengths = [train_split, test_split, len(dataset) - (train_split + test_split)]
train_dataset, test_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of test dataset:", len(test_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 6561
Length of test dataset: 1312
Length of val dataset: 875


In [8]:
def load_data(train_dataset, test_dataset, val_dataset, collate_fn):
    '''
    Arguments:
        train_dataset: train dataset of type `CustomDataset`
        test_dataset: train dataset of type `CustomDataset`
        val_dataset: validation dataset of type `CustomDataset`
        collate_fn: collate function
        
    Outputs:
        train_loader, test_loader, val_loader: train and validation dataloaders
    '''
    
    batch_size = 100
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               collate_fn=collate_fn,
                                               shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=batch_size,
                                           collate_fn=collate_fn,
                                           shuffle=False)
    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             collate_fn=collate_fn,
                                             shuffle=False)
    
    return train_loader, test_loader, val_loader


train_loader, test_loader, val_loader = load_data(train_dataset, test_dataset, val_dataset, collate_fn)

In [10]:
def indices_to_multihot(indices, masks, dim):
    multihot = torch.zeros((indices.shape[0], dim), dtype=torch.int)
    for idx, row in enumerate(indices):
        y_idx = row[masks[idx]].unique()
        multihot[idx] = F.one_hot(y_idx.to(torch.int64), multihot.shape[1]).sum(0)
    return multihot

In [18]:
class BaselineMLP(nn.Module):
    
    def __init__(self, num_codes, num_categories):
        super().__init__()
        """
        Arguments:
            num_codes: total number of diagnosis codes
            num_categories: number of higher level categories to predict
        """
        self.embedding = nn.Embedding(num_codes, embedding_dim=180, padding_idx=0)
        self.fc = nn.Linear(180, num_categories)
        
    def forward(self, x, masks):
        """
        Arguments:
            x: the diagnosis sequence of shape (batch_size, # visits, # diagnosis codes)
            masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

        Outputs:
            logits: probabilities of shape (batch_size)
        """
        x[masks] += 1  # Increment values by 1 to make use of padding_idx == 0
        x = self.embedding(x)
        x = x.sum(dim=1)
        logits = self.fc(x)
        return logits
    

# load the model here
baseline_mlp = BaselineMLP(num_codes = len(codes), num_categories=len(sub_categories))
baseline_mlp

BaselineMLP(
  (embedding): Embedding(4903, 180, padding_idx=0)
  (fc): Linear(in_features=180, out_features=184, bias=True)
)

In [19]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(baseline_mlp.parameters(), lr=0.001)
optimizer = torch.optim.Adadelta(baseline_mlp.parameters(), weight_decay=0.001)

In [20]:
def eval_model(model, test_loader, k=15, n=-1):  
    """
    Arguments:
        model: the BaselineMLP model
        test_loader: validation dataloader
        k: value for top k predictions
        n: num of records to evaluate in the batch, value -1 evaulates all records
        
    Outputs:
        precision_k: visit-level precison@k
        accuracy_k: code-level accuracy@k
    """
    y_pred = torch.LongTensor()
    y_true = torch.LongTensor()
    all_precision = []
    all_accuracy = []
    
    model.eval()
    with torch.no_grad():
        for x, masks, y in test_loader:
            n_eval = y.shape[0] - 1 if n == -1 else n
            y_hat = model(x, masks)
            y_hat = F.softmax(y_hat, dim=-1)
            nz_rows, nz_cols = torch.nonzero(y, as_tuple=True)
            k_correct = 0
            total_precision = 0
            total_accuracy = 0
            for i in range(n_eval):
                visit_correct = 0
                y_true = nz_cols[nz_rows == i]
                _, y_pred = torch.topk(y_hat[i], k)

                for v in y_true:
                    if v in y_pred:
                        visit_correct += 1
                        
                visit_precision = visit_correct / min(k, len(y_true))
                visit_accuracy = visit_correct / len(y_true)
                k_correct += visit_correct
                total_precision += visit_precision
                total_accuracy += visit_accuracy

            precision_k = total_precision / n_eval
            accuracy_k = total_accuracy / n_eval
            all_precision.append(precision_k)
            all_accuracy.append(accuracy_k)
            
    total_precision_k = np.mean(all_precision)
    total_accuracy_k = np.mean(all_accuracy)
    return total_precision_k, total_accuracy_k

In [21]:
def train(model, train_loader, test_loader, n_epochs):
    """
    Arguments:
        model: the BaselineMLP model
        train_loader: training dataloader
        test_loader: validation dataloader
        n_epochs: num epochs to train
    """
    max_cpu, max_ram = print_cpu_usage()
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, masks, y in train_loader:
            y_hat = model(x, masks)
            loss = criterion(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        cpu, ram = print_cpu_usage()
        max_cpu = cpu if cpu > max_cpu else max_cpu
        max_ram = ram if ram > max_ram else max_ram
        print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
        for k in range(5, 31, 5):
            precision_k, accuracy_k = eval_model(model, test_loader, k=k)
            print(f'Epoch: {epoch+1} \t Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')
    final_cpu, final_ram = print_cpu_usage()
    print(f"Max CPU usage: {max_cpu:.3f}\tMax RAM % usage: {max_ram}")

In [22]:
def print_cpu_usage():
    load = psutil.getloadavg()[2]
    cpu_usage = (load/os.cpu_count()) * 100
    ram = psutil.virtual_memory()[2]
    print(f"CPU: {cpu_usage:0.2f}")
    print(f"RAM %: {ram}")
    return cpu_usage, ram

In [23]:
n_epochs = 100
%time train(baseline_mlp, train_loader, val_loader, n_epochs)

CPU: 19.43
RAM %: 57.1
CPU: 19.43
RAM %: 57.1
Epoch: 1 	 Training Loss: 4.667513
Epoch: 1 	 Validation precision@k5: 0.5627, accuracy@k5: 0.2918
Epoch: 1 	 Validation precision@k10: 0.5338, accuracy@k10: 0.4476
Epoch: 1 	 Validation precision@k15: 0.5769, accuracy@k15: 0.5549
Epoch: 1 	 Validation precision@k20: 0.6368, accuracy@k20: 0.6330
Epoch: 1 	 Validation precision@k25: 0.6990, accuracy@k25: 0.6987
Epoch: 1 	 Validation precision@k30: 0.7470, accuracy@k30: 0.7470
CPU: 19.43
RAM %: 57.1
Epoch: 2 	 Training Loss: 3.860909
Epoch: 2 	 Validation precision@k5: 0.6123, accuracy@k5: 0.3186
Epoch: 2 	 Validation precision@k10: 0.5756, accuracy@k10: 0.4848
Epoch: 2 	 Validation precision@k15: 0.6197, accuracy@k15: 0.5966
Epoch: 2 	 Validation precision@k20: 0.6815, accuracy@k20: 0.6774
Epoch: 2 	 Validation precision@k25: 0.7380, accuracy@k25: 0.7377
Epoch: 2 	 Validation precision@k30: 0.7824, accuracy@k30: 0.7824
CPU: 19.64
RAM %: 56.7
Epoch: 3 	 Training Loss: 3.749623
Epoch: 3 	 Vali

CPU: 20.41
RAM %: 57.1
Epoch: 19 	 Training Loss: 3.552497
Epoch: 19 	 Validation precision@k5: 0.6712, accuracy@k5: 0.3500
Epoch: 19 	 Validation precision@k10: 0.6302, accuracy@k10: 0.5296
Epoch: 19 	 Validation precision@k15: 0.6694, accuracy@k15: 0.6442
Epoch: 19 	 Validation precision@k20: 0.7299, accuracy@k20: 0.7255
Epoch: 19 	 Validation precision@k25: 0.7857, accuracy@k25: 0.7855
Epoch: 19 	 Validation precision@k30: 0.8285, accuracy@k30: 0.8285
CPU: 20.40
RAM %: 57.1
Epoch: 20 	 Training Loss: 3.547756
Epoch: 20 	 Validation precision@k5: 0.6737, accuracy@k5: 0.3508
Epoch: 20 	 Validation precision@k10: 0.6346, accuracy@k10: 0.5336
Epoch: 20 	 Validation precision@k15: 0.6718, accuracy@k15: 0.6467
Epoch: 20 	 Validation precision@k20: 0.7325, accuracy@k20: 0.7281
Epoch: 20 	 Validation precision@k25: 0.7874, accuracy@k25: 0.7871
Epoch: 20 	 Validation precision@k30: 0.8284, accuracy@k30: 0.8284
CPU: 20.39
RAM %: 56.7
Epoch: 21 	 Training Loss: 3.543455
Epoch: 21 	 Validation 

Epoch: 36 	 Validation precision@k30: 0.8368, accuracy@k30: 0.8368
CPU: 20.36
RAM %: 57.1
Epoch: 37 	 Training Loss: 3.496924
Epoch: 37 	 Validation precision@k5: 0.6901, accuracy@k5: 0.3587
Epoch: 37 	 Validation precision@k10: 0.6537, accuracy@k10: 0.5487
Epoch: 37 	 Validation precision@k15: 0.6899, accuracy@k15: 0.6633
Epoch: 37 	 Validation precision@k20: 0.7459, accuracy@k20: 0.7413
Epoch: 37 	 Validation precision@k25: 0.7987, accuracy@k25: 0.7984
Epoch: 37 	 Validation precision@k30: 0.8383, accuracy@k30: 0.8383
CPU: 20.54
RAM %: 57.1
Epoch: 38 	 Training Loss: 3.495603
Epoch: 38 	 Validation precision@k5: 0.6905, accuracy@k5: 0.3597
Epoch: 38 	 Validation precision@k10: 0.6555, accuracy@k10: 0.5503
Epoch: 38 	 Validation precision@k15: 0.6888, accuracy@k15: 0.6623
Epoch: 38 	 Validation precision@k20: 0.7468, accuracy@k20: 0.7422
Epoch: 38 	 Validation precision@k25: 0.7985, accuracy@k25: 0.7982
Epoch: 38 	 Validation precision@k30: 0.8358, accuracy@k30: 0.8358
CPU: 20.49
RAM 

Epoch: 54 	 Validation precision@k25: 0.7984, accuracy@k25: 0.7981
Epoch: 54 	 Validation precision@k30: 0.8400, accuracy@k30: 0.8400
CPU: 21.70
RAM %: 57.1
Epoch: 55 	 Training Loss: 3.485228
Epoch: 55 	 Validation precision@k5: 0.7056, accuracy@k5: 0.3666
Epoch: 55 	 Validation precision@k10: 0.6654, accuracy@k10: 0.5589
Epoch: 55 	 Validation precision@k15: 0.6941, accuracy@k15: 0.6671
Epoch: 55 	 Validation precision@k20: 0.7468, accuracy@k20: 0.7422
Epoch: 55 	 Validation precision@k25: 0.7985, accuracy@k25: 0.7982
Epoch: 55 	 Validation precision@k30: 0.8405, accuracy@k30: 0.8405
CPU: 21.70
RAM %: 57.1
Epoch: 56 	 Training Loss: 3.484556
Epoch: 56 	 Validation precision@k5: 0.7068, accuracy@k5: 0.3683
Epoch: 56 	 Validation precision@k10: 0.6614, accuracy@k10: 0.5553
Epoch: 56 	 Validation precision@k15: 0.6943, accuracy@k15: 0.6675
Epoch: 56 	 Validation precision@k20: 0.7463, accuracy@k20: 0.7416
Epoch: 56 	 Validation precision@k25: 0.7973, accuracy@k25: 0.7970
Epoch: 56 	 Val

Epoch: 72 	 Validation precision@k20: 0.7442, accuracy@k20: 0.7396
Epoch: 72 	 Validation precision@k25: 0.7963, accuracy@k25: 0.7961
Epoch: 72 	 Validation precision@k30: 0.8386, accuracy@k30: 0.8386
CPU: 22.00
RAM %: 57.0
Epoch: 73 	 Training Loss: 3.484508
Epoch: 73 	 Validation precision@k5: 0.7032, accuracy@k5: 0.3654
Epoch: 73 	 Validation precision@k10: 0.6622, accuracy@k10: 0.5556
Epoch: 73 	 Validation precision@k15: 0.6950, accuracy@k15: 0.6681
Epoch: 73 	 Validation precision@k20: 0.7480, accuracy@k20: 0.7434
Epoch: 73 	 Validation precision@k25: 0.7960, accuracy@k25: 0.7957
Epoch: 73 	 Validation precision@k30: 0.8407, accuracy@k30: 0.8407
CPU: 22.13
RAM %: 56.9
Epoch: 74 	 Training Loss: 3.482937
Epoch: 74 	 Validation precision@k5: 0.7049, accuracy@k5: 0.3665
Epoch: 74 	 Validation precision@k10: 0.6618, accuracy@k10: 0.5551
Epoch: 74 	 Validation precision@k15: 0.6930, accuracy@k15: 0.6662
Epoch: 74 	 Validation precision@k20: 0.7452, accuracy@k20: 0.7406
Epoch: 74 	 Val

Epoch: 90 	 Validation precision@k15: 0.6941, accuracy@k15: 0.6671
Epoch: 90 	 Validation precision@k20: 0.7466, accuracy@k20: 0.7419
Epoch: 90 	 Validation precision@k25: 0.7966, accuracy@k25: 0.7963
Epoch: 90 	 Validation precision@k30: 0.8377, accuracy@k30: 0.8377
CPU: 22.63
RAM %: 57.7
Epoch: 91 	 Training Loss: 3.484789
Epoch: 91 	 Validation precision@k5: 0.7104, accuracy@k5: 0.3697
Epoch: 91 	 Validation precision@k10: 0.6609, accuracy@k10: 0.5543
Epoch: 91 	 Validation precision@k15: 0.6941, accuracy@k15: 0.6671
Epoch: 91 	 Validation precision@k20: 0.7468, accuracy@k20: 0.7422
Epoch: 91 	 Validation precision@k25: 0.7970, accuracy@k25: 0.7967
Epoch: 91 	 Validation precision@k30: 0.8389, accuracy@k30: 0.8389
CPU: 22.68
RAM %: 57.8
Epoch: 92 	 Training Loss: 3.484804
Epoch: 92 	 Validation precision@k5: 0.7039, accuracy@k5: 0.3659
Epoch: 92 	 Validation precision@k10: 0.6598, accuracy@k10: 0.5543
Epoch: 92 	 Validation precision@k15: 0.6919, accuracy@k15: 0.6651
Epoch: 92 	 Val

In [24]:
for k in range(5, 31, 5):
    precision_k, accuracy_k = eval_model(baseline_mlp, test_loader, k=k)
    print(f'Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')

Validation precision@k5: 0.6879, accuracy@k5: 0.3490
Validation precision@k10: 0.6526, accuracy@k10: 0.5466
Validation precision@k15: 0.6849, accuracy@k15: 0.6604
Validation precision@k20: 0.7417, accuracy@k20: 0.7379
Validation precision@k25: 0.7922, accuracy@k25: 0.7920
Validation precision@k30: 0.8375, accuracy@k30: 0.8375


In [1]:
torch.save(baseline_mlp, os.path.join(CHECKPOINT_PATH, "BaselineMLP_100.pth"))

NameError: name 'torch' is not defined